In [ ]:
# The code is sourced from: https://www.udemy.com/course/tigergraph/. 

In [ ]:
# Creating the vertex entity.

CREATE VERTEX entity (PRIMARY_ID entity_id INT, entity_name STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"

In [ ]:
# Creating the vertex patent.

CREATE VERTEX patent (PRIMARY_ID patent_id STRING, patent_url STRING, patent_title STRING) WITH PRIMARY_ID_AS_ATTRIBUTE = "true"

In [ ]:
# Creating the edge connected_to.

CREATE DIRECTED EDGE connected_to (FROM entity, TO entity, relation_name STRING, sentence STRING)

In [ ]:
# Creating the edge from_patent.

CREATE DIRECTED EDGE from_patent (FROM entity, TO patent)

In [ ]:
# Creating the Graph.

CREATE GRAPH SemAero (entity, patent, connected_to, from_patent)

In [ ]:
# Using the Graph.

USE GRAPH SemAero

In [ ]:
# Creating the Loading Job.

BEGIN
CREATE LOADING JOB load_SemAero FOR GRAPH SemAero {
    DEFINE FILENAME entity = "/home/tigergraph/mydata/SemAero/entity.csv";
    DEFINE FILENAME patent = "/home/tigergraph/mydata/SemAero/patent.csv";
    DEFINE FILENAME connected_to = "/home/tigergraph/mydata/SemAero/connected_to.csv";
    DEFINE FILENAME from_patent = "/home/tigergraph/mydata/SemAero/from_patent.csv";
    LOAD entity TO VERTEX entity VALUES ($"entity_id", $"entity_name") USING header = "true", separator = ",";
    LOAD patent TO VERTEX patent VALUES ($"patent_id", $"patent_url", $"patent_title") USING header = "true", separator = ",";
    LOAD connected_to TO EDGE connected_to VALUES ($"from_entity_id", $"to_entity_id", $"relation_name", $"sentence") USING header = "true", separator = ",";
    LOAD from_patent TO EDGE from_patent VALUES ($"entity_id", $"patent_id") USING header = "true", separator = ",";
}
END

In [ ]:
# Running the Loading Job.

RUN LOADING JOB load_SemAero

In [ ]:
# Creating the query get_entity to find the entity with maximum connected components.

CREATE QUERY get_entity() FOR GRAPH SemAero {
    SumAccum <INT> @num_connections;
    START = {entity.*};
    connections = SELECT a
    FROM START-(connected_to)->entity:a
    ACCUM a.@num_connections += 1
    ORDER BY a.@num_connections DESC
    LIMIT 1;
    PRINT connections;
}

In [ ]:
# Creating the query get_community to obtain the nodes connected with the given entity.

CREATE QUERY get_community(VERTEX <entity> e) FOR GRAPH SemAero {
    START = {entity.*};
    community = SELECT s
                FROM START:s-(connected_to:c)->entity:a
                WHERE a==e;
    PRINT community;
}